In [1]:
import torch
from data.dataset import SkeletonDataset
from torch_geometric.data import DataLoader
from models.net import DualGraphTransformer
from einops import rearrange
from args import make_args

In [2]:
train_dataset= SkeletonDataset(root="/home/project/gcn/Apb-gcn/NTU-RGB+D", name='cv_train', benchmark='cv', sample = 'train')

In [3]:
device = torch.device('cuda:0')

In [4]:
train_loader = DataLoader(train_dataset.data, batch_size = 4)
#val_loader = DataLoader(val_dataset, batch_size = 32)

In [5]:
model = DualGraphTransformer(in_channels = 7, hidden_channels = 16, out_channels = 16, num_layers = 4, num_heads = 8)

In [8]:
for batch_idx, batch in enumerate(train_loader):
    if batch_idx == 0:
        print(batch.x.shape)
        output = model(batch.x, adj=train_dataset.skeleton_)
        print(output.shape)
        break

torch.Size([322, 25, 7])
torch.Size([322, 60])
